In [3]:
#Preliminary Data Analysis

# Load Packages
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)

options(repr.matrix.max.rows = 6)




── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following objects are masked from ‘package:readr’:

    edition_get, local_edition


The following object is masked from ‘package:tidyr’:

    matches


── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0  

Preliminary Exploratory Data Analysis

In the code below, we can prove that the dataset can be read from the web. We will be naming the dataset "heart_data". 


In [4]:
heart_data <- read_csv("data/processed.cleveland.data", col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


We notice that there are no column names for the data set thus we will need to add column names based on what each column represents.

In [7]:
names <- c("age","sex","chest_pain","resting_bps",
           "cholestrol","fbs","restecg","thalach",
           "exang","oldpeak","slope",
           "ca","thal","diesease")

colnames(heart_data) <- names
heart_data

age,sex,chest_pain,resting_bps,cholestrol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diesease
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


Note that at the moment many column names cannot be easily understood at first glance, but for this analysis we are only interested in the variables age, chestpain type, resting blood presssure, cholesterol level, and if a patient has heart diesease or not. Therefore, let's select for the columns we are interested in and tidy it up. 

In [16]:
heart_data_select <- select(heart_data, age, chest_pain, resting_bps, cholestrol, diesease)
slice(heart_data_select,1:5)
sum(is.na(heart_data_select))

age,chest_pain,resting_bps,cholestrol,diesease
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,145,233,0
67,4,160,286,2
67,4,120,229,1
37,3,130,250,0
41,2,130,204,0


[1] 0

Using the slice function, we see that the data is tidy as each column is of only 1 variable, each row is one unique obsevation and each cell only contains one value. The is.na() functio inbedded inside the sum() function tells us how many missing values are present, which in our case is 0. 

Now, let's serperate our training data from the testing data and summarize the training idea to get a good idea of what we are working with. To make sure the results are reproducible, a seed wll be set as well. 

In [13]:
#splitting dataframe into training, testing datasets
set.seed(1)
cleveland_clean <- mutate(cleveland_data, 
                          age = as.numeric(age), cp = as.numeric(cp), 
                          chol = as.numeric(chol), trestbps = as.numeric(trestbps)) %>%
                          mutate(diag = ifelse(is.na(num), NA, (num > 0))) 

cleveland_split <- initial_split(cleveland_clean, prop = 3/4, strata = num)

cleveland_training <- training(cleveland_split)
cleveland_testing <- testing(cleveland_split)

ERROR: Error in mutate(cleveland_data, age = as.numeric(age), cp = as.numeric(cp), : object 'cleveland_data' not found
